In [14]:
import numpy as np

from gsurface.surface import Surface, SJH

# Surface definition

The goal of this notebook is to show the different way to define a new $Surface$ object.

The goal is to define the following the following surface defined with parametric equations :

$$S (u, v)=
\left[\begin{matrix}u - \cos{\left(v \right)}\\v \sin{\left(u \right)} + v\\u \sin{\left(u - v \right)} - v\end{matrix}\right]$$

In [15]:
class NewSurface(Surface):
    plimits = np.array([
        [-2.0, 2.0],
        [-2.0, 2.0]
    ])
    def _definition(self, u: float, v: float) -> SJH:        
        return self.buildMetric(
            x=u - np.cos(v),
            y=v*np.sin(u) + v,
            z=np.sin(u - v)*u - v,

            dux=1,
            duy=v*np.cos(u),
            duz=u*np.cos(u - v) + np.sin(u - v),

            dvx=np.sin(v),
            dvy=np.sin(u) + 1,
            dvz=-u*np.cos(u - v) - 1,

            dvvx = np.cos(v),

            duuy=-v*np.sin(u),
            duvy=np.cos(u),

            duuz=-u*np.sin(u - v) + 2*np.cos(u - v),
            duvz=u*np.sin(u - v) - np.cos(u - v),
            dvvz=-u*np.sin(u - v),
        )
        

Check surface consistancy

In [16]:
surface = NewSurface()

surface.check_verbose(20, 20)

=== Checking surface definition ===
U 20 points from -2.0 to 2.0
V 20 points from -2.0 to 2.0
Tolerance : 1.0E-07
index[0 : xi] max err = 4.2E-08 [True]
index[1 : yi] max err = 5.2E-08 [True]
index[2 : zi] max err = 6.5E-08 [True]
index[3 : duxi] max err = 0.0E+00 [True]
index[4 : dvxi] max err = 1.1E-08 [True]
index[5 : duyi] max err = 2.8E-08 [True]
index[6 : dvyi] max err = 1.4E-08 [True]
index[7 : duzi] max err = 5.3E-08 [True]
index[8 : dvzi] max err = 6.9E-08 [True]


True

Create model

In [17]:
from gsurface import SurfaceGuidedMassSystem, Tyi, build_s0
from gsurface.forces import ViscousFriction, SpringForce

model = SurfaceGuidedMassSystem(
    surface=surface,
    s0=build_s0(u0=2.5, du0=-8.0, dv0=5.0),
    m=1.0,
    forces=[
        SpringForce(10.0),
        ViscousFriction(1.0)
    ]
)

In [18]:
from gsurface.plotter import mayavi_plot_surfaces, SurfacePlot

surface  = NewSurface().multlims(1.5)

surface_mesh = surface.build_surface(*surface.mesh(50, 50))

time = np.linspace(0, 10, 1000)

data = model.solve(time)

solutions = model.solutions(data, time)

# animate does not work with jupyter notebooks
mayavi_plot_surfaces([
    SurfacePlot(surface_mesh, trajectory=solutions[Tyi], animate=False)
])

![New Surface Trajectory](../render/NewSurfaceTraj.png)